In [1]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [ ]:
# import IPython
# import sys
# from pygaggle.rerank.transformer import MonoT5
# !{sys.executable} -m pip install -r pygaggle/requirements.txt
# IPython.Application.instance().kernel.do_shutdown(True)  

In [2]:
""" Generate Taskgraphs from datasets """

from models_datasets.recipe_1m_model import AbstractModelDataset
from models_datasets.recipe_1m_model import Recipe1MModel
from models_datasets.wikihow_model import WikihowModel

dataset_models = {
    "COOKING" : Recipe1MModel,
    "DIY": WikihowModel,
}

for model_name, model in dataset_models.items():
    dataset_model: AbstractModelDataset = model()
    print(f"Dataset Model: {model_name}")
    dataset_model.convert_to_taskgraphs()

Dataset Model: COOKING
Taskgraphs already converted. Call overwrite=True in convert_to_taskgraphs() to overwrite the preprocessed taskgraphs.
Dataset Model: DIY
Taskgraphs already converted. Call overwrite=True in convert_to_taskgraphs() to overwrite the preprocessed taskgraphs.


In [2]:
import pandas as pd

""" Preprocess and load queries """

# from query_pipeline import query_pipeline
# query_pipeline.preprocess_queries()

queries = {
    "COOKING" : pd.read_csv('queries/cooking.csv'),
    "DIY": pd.read_csv('queries/diy.csv'),
}

queries["COOKING"].head()

,id,raw query,domain,knowledge,complex,many points of view,different key entities,seasonal,regional,target query,short query
0,query-0,how to spatchcock a turkey,cooking,n,y,n,n,y,usa,spatchcock turkey,spatchcock a turkey
1,query-1,I want an easy to make dessert for christmas,cooking,n,y,y,n,y,-,easi dessert christma,easy to make dessert for christmas
2,query-2,any recommendations for a gluten free appetizer,cooking,y,n,y,y,n,-,gluten free appet,gluten free appetizer
3,query-3,how to make a green goddess salad,cooking,y,y,n,y,n,usa,green goddess salad,green goddess salad
4,query-4,how to make a classic english trifle,cooking,y,n,n,y,y,uk,classic english trifl,classic english trifle


In [3]:
# from models_indexes.bm25_model import BM25Model
# from models_indexes.ance_model import AnceModel
# from models_indexes.colbert_model import ColbertModel
# from models_indexes.marqo_model import MarqoModel
# from models_indexes.abstract_model import AbstractModel
from models_indexes.hybrid_model import HybridModel

models = {}

for domain in  ["DIY"]:
# for domain in  ["COOKING"]:
    print(domain)
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3+t5" : BM25Model(domain = domain, rm3=True, t5=True),
        # "ance+t5": AnceModel(domain = domain, t5=True),
        # "colbert+t5": ColbertModel(domain = domain, t5=True),
        # "marqo": MarqoModel(domain = domain, t5 = True),
        # "marqo": MarqoModel(domain = domain),
        "hybrid": HybridModel(domain = domain, rm3=True, t5=True)
    }

/home/ubuntu/task-search-quality/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-12 10:29:33.403251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 10:29:37.839978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-12 10:29:37.840114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_

DIY


In [ ]:
# build indexes
print("Build indexes")
for domain, index_models in models.items():
    for model_name, model in index_models.items():
        model.build_index()

In [4]:
# create run files
print("Creating run files ")
for domain, index_models in models.items():
    qs = queries[domain]
    print(f"DOMAIN {domain}")
    for model_name, model in index_models.items():
        model.convert_search_results_to_run(qs)

Creating run files 
DOMAIN DIY
Started 1/100
converted to pygaggle


AttributeError: 'HybridModel' object has no attribute 'dense_hits_to_text'